In [1]:
import pandas as pd

In [2]:
!wget https://s3.amazonaws.com/data.patentsview.org/download/ipcr.tsv.zip

--2023-02-15 16:36:52--  https://s3.amazonaws.com/data.patentsview.org/download/ipcr.tsv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.203.232, 52.216.53.32, 52.216.179.125, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.203.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 675907728 (645M) [binary/octet-stream]
Saving to: ‘ipcr.tsv.zip’

ipcr.tsv.zip        100%[===================>] 644.60M  59.8MB/s    in 11s     

2023-02-15 16:37:04 (56.8 MB/s) - ‘ipcr.tsv.zip’ saved [675907728/675907728]



In [3]:
!unzip ./ipcr.tsv.zip

Archive:  ./ipcr.tsv.zip
  inflating: ipcr.tsv                


In [4]:
df_ipcr = pd.read_csv("./ipcr.tsv", delimiter="\t")

df_ipcr.head(5)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,uuid,patent_id,classification_level,section,ipc_class,subclass,main_group,subgroup,symbol_position,classification_value,classification_status,classification_data_source,action_date,ipc_version_indicator,sequence
0,00005z3qh82fwpo5r1oupwpr3,6864832,NaN,G,01,S,013,/42,NaN,NaN,NaN,NaN,NaN,NaN,0
1,0000662nssr53hdo3lp92sz26,9954111,A,H,01,L,27,1156,L,I,B,H,2018-04-24,2017-01-01,8
2,00008u9j3g8oivqtuc1dqayb1,10048897,A,G,06,F,12,891,L,I,B,H,2018-08-14,2016-01-01,7
3,00008v5gnw215cdjozwehxqky,10694566,A,H,4,W,4,0,F,I,B,H,2020-06-23,2018-01-01,0
4,0000hj3ytmy8g9l2qa5x1hta5,D409748,NaN,D,24,04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
df_ipcr["section"].isnull().sum()

0

In [6]:
df_ipcr["ipc_class"].isnull().sum()

0

In [7]:
df_ipcr["subclass"].isnull().sum()

15

In [8]:
df_ipcr = df_ipcr[ df_ipcr["subclass"].isnull() == False]

In [9]:
df_ipcr["ipc_class"] = df_ipcr["ipc_class"].map(str)
df_ipcr = df_ipcr[df_ipcr["ipc_class"].map(str.isdigit)]
df_ipcr["ipc_class"] = df_ipcr["ipc_class"].astype(int)

In [10]:
two_digit_f = lambda x : '{0:02d}'.format(x)
two_digit_f(3)

'03'

In [11]:
df_ipcr["ipc_class"] = df_ipcr["ipc_class"].map(two_digit_f)

In [12]:
df_ipcr["ipc_class"][:3]

0    01
1    01
2    06
Name: ipc_class, dtype: object

In [ ]:
df_ipcr["subclass"] = df_ipcr["subclass"].map(str.upper)
df_ipcr = df_ipcr[df_ipcr["subclass"].isin(list("ABCEDFGHIJKLMNOPQRSTUVWXYZ"))]

In [ ]:
df_ipcr["4digit"] = df_ipcr["section"] + df_ipcr["ipc_class"] + df_ipcr["subclass"]

In [ ]:
df_data = df_ipcr[["patent_id", "4digit"]]
df_data.describe()

In [ ]:
df_data[:5]

In [ ]:
df_data = df_data.drop_duplicates(['patent_id', '4digit'])

In [ ]:
df_data.describe()

In [ ]:
df_data["value"] = True

In [ ]:
df_small_data = df_data.loc[:50000, :]

In [ ]:
df_matrix = pd.crosstab(df_small_data.patent_id, df_small_data["4digit"], 
                        df_small_data.value, aggfunc="first")

In [ ]:
df_matrix.fillna(0)

In [ ]:
df_matrix

In [ ]:
df_matrix.loc["3930477", :].argmax()